In [0]:
from pyspark.sql.functions import col, lit, current_timestamp, concat
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

In [0]:
# What if the name was an array instead of object and contains more than 1 instance e.g. children names as an array of forename, surname... How do we handle this?

In [0]:
drivers_schema = StructType(
    [
        StructField("driverId", IntegerType(), False),
        StructField("driverRef", StringType(), True),
        StructField("number", IntegerType(), True),
        StructField("code", StringType(), True),
        StructField("name", StructType([StructField("forename", StringType(), True), StructField("surname", StringType(), True)]), True),
        StructField("dob", DateType(), True),
        StructField("nationality", StringType(), True),
        StructField("url", StringType(), True)
    ]
)

In [0]:
# Renamed Columns
# create Ingestion Column
# Create name column from forname, surname
# drop url
# save drivers to processed
# we had a column name and replaced it with the same column again in .withColumn("name", name.forename+name.surname), will it get replaced? > Yes


In [0]:
drivers_df = spark.read.json("/mnt/sanformula1dl/raw/drivers.json", schema=drivers_schema)

In [0]:
drivers_transformed_df = drivers_df.withColumnRenamed("driverId", "driver_id") \
    .withColumnRenamed("driverRef", "driver_ref") \
    .withColumn("ingestion_date", current_timestamp()) \
    .withColumn("data_source", lit("manual")) \
    .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname")))

In [0]:
driver_final_df = drivers_transformed_df.drop("url")

In [0]:
display(driver_final_df)

In [0]:
driver_final_df.write.mode("overwrite").parquet("/mnt/sanformula1dl/processed/drivers")

In [0]:
%fs
ls /mnt/sanformula1dl/processed/drivers